In [17]:
from autophrase import spacy_txt
import pandas as pd
from snapy import MinHash, LSH

In [32]:
path = "DEA"
s = spacy_txt()
doc, v, n= s.read_file(path)

gg
August 06, 2020
August 06, 2020 DATE
Contact
Phone Number
IMMEDIATE RELEASE
fentanyl-related overdose deaths
officials
public warning
Counterfeit Pills
Fentanyl
Counterfeit Pills
Fentanyl
SAN DIEGO
Law enforcement officials
the county
the public
a sharp increase
overdose deaths
the highly potent and often deadly drug
fentanyl
SAN DIEGO ORG
152 fentanyl-related overdose deaths
San Diego County
the first six months of this year
203 fentanyl-related deaths
pending confirmation
This year
deaths
all areas
the county
Victims
the average age
2019 DATE
San Diego County GPE
the first six months of this year DATE
119 CARDINAL
another 84 CARDINAL
This year DATE
age from 17 DATE
37 DATE
fentanyl-laced drugs
the DEA
San Diego County District Attorney
U.S. Attorney
San Diego County Sheriff’s Department
San Diego Police Department
San Diego County Health and Human Services
San Diego County Medical Examiner
the DEA ORG
San Diego County Sheriff’s Department ORG
San Diego Police Department ORG
San Di

ValueError: too many values to unpack (expected 3)

In [ ]:
noun = doc[0].sentences[0].noun
ner = doc[0].sentences[0].ner

In [20]:
mesh_fp = 'MESH.csv'
rxnorm_fp = 'RXNORM.csv'
rx=pd.read_csv(rxnorm_fp,usecols=['Preferred Label','Semantic type UMLS property'])
mesh=pd.read_csv(mesh_fp,usecols=['Preferred Label','Semantic type UMLS property'])
term = pd.concat([rx,mesh])
term['Preferred Label'] = term['Preferred Label'].str.lower()

In [31]:
ontology = dict(zip(term['Preferred Label'],term['Semantic type UMLS property']))

In [46]:
list(ontology.keys())[0]

'fentanyl 0.3 mg buccal tablet'

In [39]:
def make_content_all(search_terms, sentences):
    #Initialize a dictionary
    content = dict()
    
    #Add search terms to dictionary with numbered key
    for i in range(len(search_terms)):
        if len(search_terms[i]) > 3:
            content[i] = search_terms[i].strip().lower()
                
    #Add text to dictionary with author as key
    for i in range(len(sentences)):
        key_n = 'sentence%d' % (i)
        for ls in sentences[i]:
            count = 0
            for n in ls:
                n = n.lower()
                if len(n) > 3:
                    key = key_n+'_'+str(count)
                    content[key] = n
                    count += 1
    return content

In [40]:
def create_lsh(content, n_permutations, n_gram):
    labels = content.keys()
    values = content.values()
    #Create MinHash object
    minhash = MinHash(values, n_gram=n_gram, permutations=n_permutations, hash_bits=64, seed=3)
    #Create LSH model
    lsh = LSH(minhash, labels, no_of_bands=5)
    return lsh


In [ ]:
def add_similar_edges(lsh, content):
    graph=nx.DiGraph()
    #For all items in content
    ls = {}
    for index, text in content.items():        
        #If item is one of the search terms
        if type(index) == int:
            q = lsh.query(index, min_jaccard=0.2)
            
            #For all matches found
            for match in q:
                
                #If matched item is from the reddit text
                if type(match) != int:
                    author = match.split("_")[0]
                    if author not in ls.keys():
                        ls[author] = [text]
                    else:
                        ls[author].append(text)
    for i in ls:
        ls[i]=list(set(ls[i]))
        total=len(ls[i])
        for n in range(total):
            for j in list(range(total))[n+1:]:
                graph.add_edge(ls[i][n],ls[i][j],author_id=i)
    return graph